In [2]:
# 필요한 라이브러리 임포트
import requests                      # HTTP 요청을 위한 라이브러리
from PIL import Image                # 이미지 처리를 위한 Pillow 라이브러리
import gradio as gr                  # 웹 인터페이스 생성을 위한 Gradio
import io                            # 바이너리 데이터 처리를 위한 입출력 모듈

# Azure Custom Vision Prediction 키와 엔드포인트 URL
PREDICTION_KEY = "48fBztGiqcoH4PUUK23qmvxTBMmu31mgQ4JmJN1EDeCWNYw2QAtQJQQJ99BEACL93NaXJ3w3AAAIACOGelyc"
ENDPOINT_URL = "https://neptun91cv20250526-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/175c111e-1670-4408-84d6-0e68daf1729c/classify/iterations/Iteration1/image"

# API 요청 헤더 설정
headers = {
    "Prediction-Key": PREDICTION_KEY,             # 인증을 위한 Prediction 키
    "Content-Type": "application/octet-stream"    # 전송하는 데이터 형식 지정 (바이너리 이미지)
}

# 예측 함수 정의 (Gradio에 연결할 함수)
def predict_with_api(image: Image.Image):
    # 이미지를 바이너리로 변환
    buf = io.BytesIO()                  # 메모리 내 바이트 버퍼 생성
    image.save(buf, format='JPEG')      # 이미지를 JPEG 형식으로 저장
    byte_data = buf.getvalue()          # 버퍼에서 바이너리 데이터 추출
    
    # Azure Custom Vision API에 POST 요청
    response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)
    
    # API 응답에서 예측 결과 추출 (JSON 형식)
    predictions = response.json()["predictions"]
    
    # 확률이 높은 상위 2개 결과만 선택 (내림차순 정렬)
    top_predictions = sorted(predictions, key=lambda x: x["probability"], reverse=True)[:2]
    
    # 결과 문자열 포맷팅
    result_lines = [
        f"{pred['tagName']} ({pred['probability'] * 100:.2f}%)"  # 예: "cat (97.23%)"
        for pred in top_predictions
    ]
    
    return "\n".join(result_lines)  # 여러 줄의 문자열로 반환

# Gradio 인터페이스 정의
interface = gr.Interface(
    fn=predict_with_api,                 # 호출할 함수
    inputs=gr.Image(type="pil"),         # 사용자 입력: 이미지 (PIL 형식)
    outputs=gr.Text(),                   # 출력 형식: 텍스트
    title="Custom Vision Image Classifier",   # 웹 앱 제목
    description="Upload an image to see the prediction from your Custom Vision model."  # 설명 텍스트
)

# Gradio 웹 인터페이스 실행
interface.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
